<a href="https://colab.research.google.com/github/karaage0703/colab-notebooks/blob/main/openai_billing_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Billing Insights

References:

- https://github.com/KazuyaIriguchi/openai-billing-insights

License:

same as https://github.com/KazuyaIriguchi/openai-billing-insights

Import library

In [ ]:
import requests
from datetime import datetime, timedelta
from collections import defaultdict
import matplotlib.pyplot as plt

Input open ai key

In [ ]:
OPEN_API_KEY = ''

Input check date

In [ ]:
# today example
check_date = datetime.today().date()

# other day
# check_date = '2023-08-01'

Define model price

Reference: https://openai.com/pricing

In [ ]:
model_pricing = {
  "gpt-3.5-turbo-0301": {
    "context": 0.0015,
    "generated": 0.002,
    "per_tokens": 1000
  },
  "gpt-3.5-turbo-0613": {
    "context": 0.0015,
    "generated": 0.002,
    "per_tokens": 1000
  },
  "gpt-3.5-turbo-16k": {
    "context": 0.003,
    "generated": 0.004,
    "per_tokens": 1000
  },
  "gpt-3.5-turbo-16k-0613": {
    "context": 0.003,
    "generated": 0.004,
    "per_tokens": 1000
  },
  "gpt-4-0314": {
    "context": 0.03,
    "generated": 0.06,
    "per_tokens": 1000
  },
  "gpt-4-0613": {
    "context": 0.03,
    "generated": 0.06,
    "per_tokens": 1000
  },
  "gpt-4-32k": {
    "context": 0.06,
    "generated": 0.12,
    "per_tokens": 1000
  },
  "gpt-4-32k-0314": {
    "context": 0.06,
    "generated": 0.12,
    "per_tokens": 1000
  },
  "gpt-4-32k-0613": {
    "context": 0.06,
    "generated": 0.12,
    "per_tokens": 1000
  },
  "whisper-1": {
    "context": 0.006,
    "generated": 0,
    "per_minutes": 1
  },
  "text-embedding-ada-002-v2": {
    "context": 0.0001,
    "generated": 0,
    "per_tokens": 1000
  },
  "text-davinci:001": {
    "context": 0.02,
    "generated": 0.02,
    "per_tokens": 1000
  },
  "text-davinci:002": {
    "context": 0.02,
    "generated": 0.02,
    "per_tokens": 1000
  },
  "text-davinci:003": {
    "context": 0.02,
    "generated": 0.02,
    "per_tokens": 1000
  }
}

Define function

In [ ]:
def call_usage_api(date: str) -> requests.Response:
    """一日の使用状況を取得

    Args:
        date (str): 日付(YYYY-mm-dd)

    Returns:
        requests.Response: レスポンス
    """
    # エンドポイントとヘッダーの設定
    url = "https://api.openai.com/v1/usage"
    headers = {"Authorization": f"Bearer {OPEN_API_KEY}"}
    params = {"date": date}

    try:
        # APIリクエストの実行
        response = requests.get(url, headers=headers, params=params)
        return response
    except Exception as e:
        print(e)
        return e

def get_model_costs(data: dict) -> dict:
    """GPTモデルのコストを取得

    Args:
        data (dict): 一日の使用状況データ

    Returns:
        dict: GPTモデルごとのコスト
    """
    # 各モデルの統計を保存する辞書
    model_stats = defaultdict(lambda: {'n_context_tokens_total': 0, 'n_generated_tokens_total': 0})

    # データの各エントリをループして、各モデルの統計を計算
    for entry in data['data']:
        snapshot_id = entry['snapshot_id']
        model_stats[snapshot_id]['n_context_tokens_total'] += entry['n_context_tokens_total']
        model_stats[snapshot_id]['n_generated_tokens_total'] += entry['n_generated_tokens_total']

    # 各モデルのコストを計算
    model_costs = {}
    for model, stats in model_stats.items():
        try:
            cost_rate = model_pricing[model]
            context_token_cost = stats['n_context_tokens_total'] * (cost_rate['context'] / cost_rate["per_tokens"])
            generated_token_cost = stats['n_generated_tokens_total'] * (cost_rate['generated'] / cost_rate["per_tokens"])
            total_cost = context_token_cost + generated_token_cost
            model_costs[model] = {
                'context_token_cost': context_token_cost,
                'generated_token_cost': generated_token_cost,
                'total_cost': total_cost
           }
        except:
            print('error in ' + model)
    model_costs["total_costs"] = sum(v["total_cost"] for k, v in model_costs.items())
    return model_costs

def get_whisper_costs(data: dict) -> dict:
    """Whisperのコストを取得

    Args:
        data (dict): 一日の使用状況データ

    Returns:
        dict: Whisperのコストデータ
    """
    whisper_data = data.get("whisper_api_data", [])
    # 合計使用秒数とコストを計算
    whisper_cost_def = model_pricing["whisper-1"]
    total_seconds = sum(entry["num_seconds"] for entry in whisper_data)
    total_whisper_cost = (total_seconds / (whisper_cost_def["per_minutes"] * 60)) * whisper_cost_def["context"]
    return {
        "total_seconds": total_seconds,
        "total_costs": total_whisper_cost
    }

def get_daily_usage(date: str) -> dict:
    """一日の使用状況、コストを取得

    Args:
        date (str): 日付(YYYY-mm-dd)

    Returns:
        dict: 使用状況・コスト
    """
    # コスト取得
    data = call_usage_api(date).json()

    # GPTモデル、Whisperのコスト計算
    # todo: fine tuning, dalleモデルへの対応
    model_costs = get_model_costs(data)
    whisper_costs = get_whisper_costs(data)
    return {
        "total_costs": model_costs["total_costs"] + whisper_costs["total_costs"],
        "model_costs": model_costs,
        "whisper_costs": whisper_costs
    }

Calculate costs

In [ ]:
data = get_daily_usage(check_date)
models = list(data['model_costs'].keys())
costs = [data['model_costs'][model] if model != 'total_costs' else data['total_costs'] for model in models]
total_costs = [cost['total_cost'] if isinstance(cost, dict) else cost for cost in costs]

Visualization

In [ ]:
plt.figure(figsize=(10, 6))
bars = plt.bar(models, total_costs, color='blue')

# Add data labels on top of each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.5, round(yval, 2), ha='center', va='bottom', fontsize=9)

plt.xlabel('Models')
plt.ylabel('Total Cost($)')
plt.title('Costs of Different Models')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.show()